# Coherencia de clusters

Tras haber descubierto las posibles estructuras en nuestro conjunto de datos, vamos a pasar por un modelo de clasificación supervisada para comprobar la coherencia de los clusters encontrados. Nos decantamos por un Random Forest.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pickle import dump, load

In [10]:
xtest = pd.read_csv('../../Samples/Clean/Feature_Selection/X_test.csv')
xtrain = pd.read_csv('../../Samples/Clean/Feature_Selection/X_train.csv')
ytest = pd.read_csv('../../Samples/Clean/Feature_Selection/Y_test.csv')
ytrain = pd.read_csv('../../Samples/Clean/Feature_Selection/Y_train.csv')

In [16]:
df = pd.read_csv('../../Samples/Clean/Testing/HDBSCAN/X_data_with_y_predict.csv')
y_hdbscan = df['cluster_hdbscan']
x_full = df.drop(columns=['cluster_hdbscan']).copy(deep=True)

X_train, X_test, y_train, y_test = train_test_split(x_full, y_hdbscan, test_size=0.2, random_state=42)

In [17]:
df.cluster_hdbscan.unique()

array([ 1,  3,  2,  0, -1])

In [22]:
n_cluster = len(y_hdbscan.unique().tolist())
rf = RandomForestClassifier(n_estimators=n_cluster, random_state=42, min_samples_split=300000)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

/home/taticc/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/taticc/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/taticc/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

          -1       0.75      1.00      0.86     86234
           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00     28396
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00        48

    accuracy                           0.75    114685
   macro avg       0.15      0.20      0.17    114685
weighted avg       0.57      0.75      0.65    114685

Accuracy: 0.7519204778305794


In [19]:
y_pred_full = rf.predict(x_full)
coherencia = np.mean(y_pred_full == y_hdbscan)
print(f"Coherencia: {coherencia * 100:.2f}%")

Coherencia: 100.00%


In [ ]:
df_final = df.copy(deep = True)
df_final['cluster_RF'] = y_pred_full

In [ ]:
df_final.to_csv('../../Samples/Clean/Final/STARTSOutput.csv', index=False)
dump(rf, open("../../Models/RF_nestimtors-"+str(n_cluster)+"_criterion-gini_maxdepht-none_minsamplessplit-20_minsamplesleaf-1_minweight-0_maxfeatures-auto_maxleafnodes-none_randomstates-42.pkl", "wb"))